In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from astropy.table import Table
import gravity_obs.p2_tools as p2t

In [3]:
table_path = '../../gvprep/tabs/star_k12mag_grp10-12mag_gaia.ipac'
tb = Table.read(table_path, format='ascii.ipac')

In [4]:
tb

name,ra_2000_hms,dec_2000_dms,ra_2000,dec_2000,H,K,parallax,pm,pmra,pmdec,radial_velocity,Grp,G,ra_2016,dec_2016,source_id
,,,,,mag,mag,mas,mas / yr,mas / yr,mas / yr,km / s,mag,mag,deg,deg,
str28,str12,str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64
Gaia DR3 5142488558250979328,01:46:31.941,-16:22:37.834,26.633086526145192,-16.37717622886039,12.06499958,12.052000046,0.452,5.476,-0.53,-5.45,--,11.887556,11.805489,26.63308407318,-16.37720045152,5142488558250979328
Gaia DR3 5084896688945791232,03:33:14.647,-25:52:17.978,53.3110285533144,-25.87166045774542,12.288999557,12.37100029,2.5197,26.985,1.777,26.927,--,11.620219,11.249787,53.31103732933,-25.87154078439,5084896688945791232
Gaia DR3 4883474679951350656,04:01:05.312,-32:23:46.026,60.2721352444382,-32.396118251637056,11.937000275,12.017000198,5.4289,32.76,32.688,-2.17,--,11.416053,11.149598,60.2723073016,-32.39612789606,4883474679951350656
Gaia DR3 4891229020066458496,04:29:08.372,-28:47:33.903,67.28488536199974,-28.792750929792273,12.074999809,12.121999741,0.2499,2.084,0.249,-2.069,--,11.761063,11.601466,67.2848866249,-28.79276012627,4891229020066458496
Gaia DR3 3171401381762257280,04:31:51.983,-17:16:45.626,67.96659695305551,-17.279340419978528,12.045999527,12.015999794,0.2479,2.938,2.897,-0.487,--,11.866324,11.751601,67.96661043766,-17.27934258484,3171401381762257280
Gaia DR3 6855176517802163456,20:43:46.809,-22:10:30.073,310.94503816160056,-22.175020399271638,9.2930002213,12.027000427,7.0276,39.24,22.188,-32.365,-16.24,10.437988,11.375767,310.94514465336,-22.17516424321,6855176517802163456
Gaia DR3 6805917533587127936,20:51:29.867,-24:09:05.141,312.8744459313909,-24.151428127987316,12.133000374,12.142000198,0.213,11.721,-4.772,-10.705,--,11.865927,11.740895,312.87442268965,-24.15147570763,6805917533587127936
Gaia DR3 6624328240456787840,22:38:54.217,-23:52:20.921,339.7259045265013,-23.872478140354218,12.25399971,12.296999931,0.1713,8.156,4.06,-7.074,--,11.90729,11.719517,339.72592425944,-23.87250957945,6624328240456787840


In [5]:
tb.add_index('name')

In [6]:
from gravity_obs import ob_add_target
def single_off_target(grav, target, folder_name=None,
                         usename=False, verify=False):
    ao_mode = "LGS_VIS"
    tel_alpha = target['ra_2000_hms']
    tel_delta = target['dec_2000_dms']

    if usename:
            obname = f"{ao_mode}_{target['name']}".replace(' ', '_')
    else:
            obname = f"{ao_mode}_{tel_alpha.split('.')[0].replace(':', '')}{tel_delta.split('.')[0].replace(':', '')}_{target['Grp']:4.1f}_{target['K']:4.1f}".replace(' ', '_')


    ob, obV = grav.create_OB(obname, folder_name=folder_name, overwrite=True)
    obsDes = ob['obsDescription']
    obsDes['name'] = target['name']
    obsDes['userComments'] = 'Single on-axis observation test'

    ob_add_target(ob, target['name'], 
                    ra = target['ra_2000_hms'], 
                    dec = target['dec_2000_dms'],
                    pma = target['pmra']/1e3,
                    pmd = target['pmdec']/1e3)
    ob, obV = grav.save_OB(obname, folder_name=folder_name)


    dit, ndit = 3, 40
    grav.add_GRAVITY_single_offaxis_acq(name=obname, folder_name=folder_name,
                                        sc_name=target['name'],
                                        sc_kmag=target['K'],
                                        sc_hmag=target['H'],
                                        ngs_mag=target['Grp'],
                                        ao_mode=ao_mode,
                                        baseline=['UTs'])
    grav.add_GRAVITY_single_obs_exp(obname, folder_name=folder_name,
                                    dit=dit, ndit_obj=ndit, ndit_sky=ndit,)

    # optional verification
    if verify:
        obID = ob['obId']
        response, _ = grav.api.verifyOB(obID,True)
        if response['observable']:
            print('*** Congratulations. Your OB' , obID, ob['name'], 'is observable!')
        else:
            print('OB', obID, 'is >>not observable<<. See messages below.')
            print(' ', '\n  '.join(response['messages']))

In [7]:
grav = p2t.p2api_GRAVITY('116.29H7.001', 'GPLUSTeam', 'GPAOwFRINGES', no_warning=True)
root_folder = 'TTR-116.0046 October First Light'
grav.set_rootFolder(root_folder)

In [8]:
target_names = ["Gaia DR3 5142488558250979328", "Gaia DR3 6624328240456787840"]
folder_name = 'offaxis_ob_test'
grav.create_folder(folder_name, overwrite=True)

for obj_name in target_names:
    target = tb.loc[obj_name]
    single_off_target(grav, target, folder_name=folder_name)